Let's start with the basics.

# Connecting to Spark

We're going to be using Python with DataFrames, which is only available in Spark 1.3 or later.  We're going to be using a recent version of open source spark.  To use it, you'll have to import the `SQLContext`.

In [ ]:
from pyspark.sql import SQLContext
sql = SQLContext(sc)

Let's set up some common functions

# Reading a table

In [ ]:
user = sql.read.format("org.apache.spark.sql.cassandra").\
               load(keyspace="users", table="user")

# Display

In [ ]:
user.collect()

# Basic Filtering

In [ ]:
user[user.age > 35].collect()

In [ ]:
user[user.age < 20 and user.name.startswith("Baby") ].collect()

# A nicer reader

In [ ]:
def create_reader(sql):
    def reader(keyspace, table):
        df = sql.read.format("org.apache.spark.sql.cassandra").\
               load(keyspace=keyspace, table=table)
        return df
    return reader

reader = create_reader(sql)

In [ ]:
apd.collect()

# A Nicer Writer

In [ ]:
def create_writer(sql, mode="append"):
    def writer(df, keyspace, table):
        df.write.format("org.apache.spark.sql.cassandra").\
                 options(table=table, keyspace=keyspace).save(mode="append")
    return writer

writer = create_writer(sql)

# Aggregations

In [ ]:
user = reader("users", "user")

# Working with Collections

# Migrating to a new structure

In [ ]:
from pyspark.sql.functions import *
result = apd.select(explode(apd.favorite_foods).alias("food"), "user_id")
writer(result, "users", "favorite_foods_index")

# SparkSQL

Register dataframe as a table

do whatever you want with it

In [ ]:
user.registerTempTable("user")

In [ ]:
sql.sql("select * from user where age > 15").collect()

# Load the movie lens data

Dataset lives in ml-10M100K directory


In [ ]:
movies = sc.textFile("ml-10M100K/movies.dat").map(lambda x: x.split("::") )

In [ ]:
movies = movies.map(lambda (x,y,z): (x,y,z.split("|")))

In [ ]:
movies = movies.toDF(["movie_id", "name", "tags"])

In [ ]:
movies.collect()